In [2]:
import pandas as pd

In [3]:
url = '../mail.csv'
dtype={'To':'string', 'Subject':'string', 'X-From':'string',
       'X-To':'string', 'X-cc':'string', 'X-bcc':'string', 'X-Folder':'string', 'X-Origin':'string', 'X-FileName':'string',
       'content':'string', 'user':'string', 'Cat_1_level_1':'string', 'Cat_1_level_2':'string', 'Cat_1_weight':'string',
       'Cat_2_level_1':'string', 'Cat_2_level_2':'string', 'Cat_2_weight':'string', 'Cat_3_level_1':'string',
       'Cat_3_level_2':'string', 'Cat_3_weight':'string', 'Cat_4_level_1':'string', 'Cat_4_level_2':'string',
       'Cat_4_weight':'string', 'Cat_5_level_1':'string', 'Cat_5_level_2':'string', 'Cat_5_weight':'string',
       'Cat_6_level_1':'string', 'Cat_6_level_2':'string', 'Cat_6_weight':'string', 'Cat_7_level_1':'string',
       'Cat_7_level_2':'string', 'Cat_7_weight':'string', 'Cat_8_level_1':'string', 'Cat_8_level_2':'string',
       'Cat_8_weight':'string', 'Cat_9_level_1':'string', 'Cat_9_level_2':'string', 'Cat_9_weight':'string',
       'Cat_10_level_1':'string', 'Cat_10_level_2':'string', 'Cat_10_weight':'string', 'Cat_11_level_1':'string',
       'Cat_11_level_2':'string', 'Cat_11_weight':'string', 'Cat_12_level_1':'string', 'Cat_12_level_2':'string',
       'Cat_12_weight':'string'}
data = pd.read_csv(url,dtype=dtype)

In [6]:
import string 

#data
#sum([i.strip(string.punctuation).isalpha() for i in data['content'][2].split()]) 

# Creation d'une colonne nombre de caracteres

In [7]:
data['content_char_len'] = data['content'].apply(lambda x: len(x)  if (type(x)==type('str')) else 0)

In [8]:
#data.boxplot(column=['content_len'])
#Boxplot inexploitable

In [9]:
data['content_char_len'].describe()

count    100000.00000
mean       2158.93104
std        7370.24427
min           0.00000
25%         272.00000
50%         758.00000
75%        1854.00000
max      260461.00000
Name: content_char_len, dtype: float64

In [ ]:
## Creating quantiles

In [10]:
content_char_length_q1 = data['content_char_len'].quantile(0.25)
content_char_length_q2 = data['content_char_len'].quantile(0.75)
content_char_length_q95 = data['content_char_len'].quantile(0.95)

In [11]:
data['content_char_length_category'] = data['content_char_len'].apply(lambda x: 'short message' if x<content_char_length_q1 else ('average message' if x<content_char_length_q2 else 'long message'))

In [12]:
data['content_char_length_category']

0          short message
1        average message
2          short message
3          short message
4          short message
              ...       
99995    average message
99996    average message
99997    average message
99998       long message
99999    average message
Name: content_char_length_category, Length: 100000, dtype: object

# Creation d'une colonne nombre de mots

In [13]:
data['content_word_len'] = data['content'].apply(lambda x: sum([i.strip(string.punctuation).isalpha() for i in x.split()])  if (type(x)==type('str')) else 0)

In [14]:
data['content_word_len'].describe()

count    100000.00000
mean        296.20993
std        1034.19512
min           0.00000
25%          38.00000
50%         105.00000
75%         253.00000
max       38084.00000
Name: content_word_len, dtype: float64

In [ ]:
## Creating quantiles

In [28]:
content_word_length_q1 = data['content_word_len'].quantile(0.25)
content_word_length_q2 = data['content_word_len'].quantile(0.75)
content_word_length_q95 = data['content_word_len'].quantile(0.95)

929.0


In [16]:
data['content_word_length_category'] = data['content_word_len'].apply(lambda x: 'short message' if x<content_word_length_q1 else ('average message' if x<content_word_length_q2 else 'long message'))

# Getting the top 5 longest messages

In [17]:
data = data.sort_values("content_word_len",ascending=False)

In [18]:
data["content_word_len"].head(5)

75868    38084
56264    38084
74539    35397
58238    35397
73895    32437
Name: content_word_len, dtype: int64

# Cleaning the 'from' field

In [19]:
data["From"][1500]

"frozenset({'david.oxley@enron.com'})"

In [43]:
def clean_from_attribute (field):
    return field[field.find("{")+1:field.find("}")][1:-1]


In [44]:
 clean_from_attribute(data["From"][1500])

'david.oxley@enron.com'

In [47]:
data['From'] = data['From'].apply(lambda x: clean_from_attribute(x))

# Cleaning 'To' attribute

In [4]:
data['To']

0              frozenset({'tim.belden@enron.com'})
1           frozenset({'john.lavorato@enron.com'})
2            frozenset({'leah.arsdall@enron.com'})
3             frozenset({'randall.gay@enron.com'})
4              frozenset({'greg.piper@enron.com'})
                           ...                    
99995    frozenset({'carlos.rodriguez@enron.com'})
99996       frozenset({'stella.morris@enron.com'})
99997        frozenset({'aimee.lannou@enron.com'})
99998          frozenset({'pat.clynes@enron.com'})
99999        frozenset({'julie.meyers@enron.com'})
Name: To, Length: 100000, dtype: string

In [ ]:
def clean_to_attribute (field):
    if type(field).__name__ == "NAType":
        return "NA"
    else:
        return field[field.find("{")+1:field.find("}")][1:-1].split(", ")

### Problème de gestion des endroits où les données sont NA, sinon créer une liste de mails

In [95]:
data['To'] = data['To'].apply(lambda x: clean_to_attribute(x))

AttributeError: 'list' object has no attribute 'find'

In [7]:
clean_to_attribute(data["To"][0])

['tim.belden@enron.com']

# Counting the number of receivers
## Il suffit juste de faire len(attribut), attribut étant l'attribut des listes d'emails

In [11]:
len(clean_to_attribute(data["To"][0]))

1